## Librerias

In [ ]:
random_state = 43992294

import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
import seaborn as sns
from scipy.stats import chi2_contingency
import scipy.stats as ss
import os
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MultiLabelBinarizer

from xgboost import XGBClassifier
from sklearn.preprocessing import FunctionTransformer

import ast

import warnings

from tqdm.auto import tqdm

## Funciones

In [2]:
def limpiar_y_agrupar(df, columna, umbral=100):
    """
    Limpia y agrupa categorías en una columna específica de un DataFrame.
    Las categorías con frecuencia menor que el umbral se agrupan en 'Otros_{columna}',
    y los valores NaN se reemplazan por 'Desconocidos_{columna}'.

    Parameters:
    - df: DataFrame de pandas.
    - columna: Nombre de la columna a limpiar y agrupar.
    - umbral: Número mínimo de observaciones para mantener una categoría.

    Returns:
    - df: DataFrame con la columna modificada.
    """
    # Contar la frecuencia de cada categoría incluyendo NaN
    conteo_categorias = df[columna].value_counts(dropna=False)
    
    # Identificar categorías con observaciones por debajo del umbral
    categorias_pequenas = conteo_categorias[conteo_categorias < umbral].index

    # Filtrar las categorías poco frecuentes excluyendo NaN
    categorias_pequenas = [cat for cat in categorias_pequenas if pd.notna(cat)]
    
    # Crear valor de relleno para categorías poco frecuentes
    valor_pequeno = f'Otros'

    # Crear valor de relleno para NaNs
    valor_desconocido = f'Desconocidos'

    # Aplicar la asignación de categorías
    df[columna] = df[columna].apply(
        lambda x: valor_pequeno if x in categorias_pequenas 
                  else (valor_desconocido if pd.isna(x) else x)
    )
    
    return df

def cramers_v(confusion_matrix):
    """
    Calcula la estadística de Cramér's V para medir la asociación entre dos variables categóricas.

    Parameters:
    - confusion_matrix: Matriz de confusión (tabla de contingencia) entre dos variables.

    Returns:
    - Cramér's V: Valor entre 0 y 1 que indica la fuerza de la asociación.
    """
    # Calcular el estadístico chi-cuadrado
    chi2 = chi2_contingency(confusion_matrix, correction=False)[0]
    # Número total de observaciones
    n = confusion_matrix.sum().sum()
    # Obtener el número de filas y columnas de la matriz de confusión
    r, k = confusion_matrix.shape
    if min(r, k) == 1:
        return np.nan  # Evitar dividir por cero
    # Calcular Cramér's V
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))


def expand_list_dummies(df, column, delimiter='|', prefix=None, suffix=None):
    """
    Expande una columna que contiene listas (o cadenas representando listas) en múltiples columnas binarias
    utilizando pandas.get_dummies, una por cada categoría única.

    Parameters:
    - df (pd.DataFrame): DataFrame de pandas.
    - column (str): Nombre de la columna a expandir.
    - delimiter (str): Delimitador a usar en get_dummies. Por defecto es '|'.
    - prefix (str, optional): Prefijo para las nuevas columnas binarias.
    - suffix (str, optional): Sufijo para las nuevas columnas binarias.

    Returns:
    - categories (set): Conjunto de categorías únicas encontradas.
    - df_expanded (pd.DataFrame): DataFrame con las nuevas columnas binarias añadidas y la columna original eliminada.
    """
    # Copiar el DataFrame para evitar modificar el original
    df = df.copy()
    
    # Definir el valor para valores faltantes
    valor_desconocido = f'Desconocidos_{column}'
    
    # Reemplazar NaN por el valor desconocido
    df[column] = df[column].fillna(f'["{valor_desconocido}"]')
    
    # Convertir las cadenas que representan listas en listas reales de Python
    def parse_list(x):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return parsed
            else:
                return [x]
        except (ValueError, SyntaxError):
            return [x]
    
    df[column] = df[column].apply(parse_list)
    
    # Convertir listas en cadenas separadas por el delimitador
    df['temp_combined'] = df[column].apply(lambda lst: delimiter.join(map(str, lst)))
    
    # Generar las columnas binarias usando get_dummies
    dummies = df['temp_combined'].str.get_dummies(sep=delimiter)
    
    # Añadir prefijo y/o sufijo si se especifica
    if prefix:
        dummies = dummies.add_prefix(f"{prefix}_")
    if suffix:
        dummies = dummies.add_suffix(f"_{suffix}")
    
    # Concatenar las columnas binarias al DataFrame original
    df_expanded = pd.concat([df.drop(columns=[column, 'temp_combined']), dummies], axis=1)
    
    # Obtener las categorías únicas
    categories = set(dummies.columns)
    
    return categories, df_expanded


def categorizar_hora(hora):
    """
    Categoriza una hora dada en 'mañana', 'tarde' o 'noche'.

    Parameters:
    - hora: Cadena de tiempo en formato 'HH:MM:SS'.

    Returns:
    - Categoría de la parte del día: 'mañana', 'tarde' o 'noche'.
    """
    # Extraer la hora como entero
    hora = int(hora.split(':')[0])
    if 0 <= hora < 12:
        return 'mañana'
    elif 12 <= hora < 18:
        return 'tarde'
    else:
        return 'noche'


def convertir_auction_time(df):
    """
    Convierte la columna 'auction_time' de formato timestamp a categorías de parte del día ('mañana', 'tarde', 'noche')
    y crea columnas binarias para cada categoría.

    Parameters:
    - df: DataFrame de pandas.

    Returns:
    - Lista de categorías creadas.
    - df: DataFrame con las nuevas columnas binarias añadidas y las columnas originales eliminadas.
    """
    # Convertir 'auction_time' de timestamp a cadena de tiempo 'HH:MM:SS'
    df['auction_time'] = pd.to_datetime(df['auction_time'], unit='s').dt.strftime('%H:%M:%S')
    
    # Crear una nueva columna con las categorías de parte del día
    df['parte_del_dia'] = df['auction_time'].apply(categorizar_hora)
    
    # Crear columnas binarias para cada parte del día
    df['mañana'] = (df['parte_del_dia'] == 'mañana').astype(int)
    df['tarde'] = (df['parte_del_dia'] == 'tarde').astype(int)
    df['noche'] = (df['parte_del_dia'] == 'noche').astype(int)
    
    # Eliminar las columnas intermedias si no se necesitan
    df.drop(['parte_del_dia', 'auction_time'], axis=1, inplace=True)
    
    return ['mañana', 'tarde', 'noche'], df


def IntFloatToStr(df):
    """
    Limpia los datos del DataFrame convirtiendo cualquier columna que contenga una combinación de 
    strings con enteros o strings con floats a cadenas de texto. Los valores NaN se reemplazan por '0'.

    Parameters:
    - df: DataFrame de pandas.

    Returns:
    - df: DataFrame limpio con las columnas convertidas a strings donde corresponda.
    """
    # Recorrer todas las columnas del DataFrame
    for column in tqdm(df.columns, desc="Limpiando datos"):
        unique_types = df[column].apply(type).unique()
        
        # Verificar si la columna contiene tanto strings como enteros o floats
        if {str, int}.issubset(set(unique_types)) or {str, float}.issubset(set(unique_types)):
            # Convertir todos los valores a string, reemplazando NaN por '0'
            df[column] = df[column].apply(lambda x: str(x) if pd.notna(x) else '0')
    
    return df


def data_cleaning(df):
    """
    Realiza la limpieza y preprocesamiento de un DataFrame siguiendo varios pasos:
    - Convertir y expandir columnas específicas.
    - Agrupar categorías poco frecuentes en columnas categóricas.
    - Limpiar los datos finales.

    Parameters:
    - df: DataFrame de pandas.

    Returns:
    - all_categories: Conjunto de todas las categorías únicas encontradas.
    - df: DataFrame limpio y preprocesado.
    """

    df = df.drop(columns=['device_id'])
    
    all_categories = set()
    
    # Mostrar el tamaño inicial del DataFrame
    print(f"Paso inicial: Tamaño del DataFrame: {df.shape}")

    # Identificar columnas categóricas excluyendo algunas específicas
    categorical_features = [
        col for col in df.select_dtypes(include=['object']).columns.tolist()
        if col not in ['auction_list_0', 'action_list_1', 'action_list_2', 'auction_time']
    ]
    print(f"Paso 1: Columnas categóricas identificadas (excluidas algunas): {categorical_features}")

    # Agrupar categorías poco frecuentes en las columnas categóricas
    for col in tqdm(categorical_features, desc="Agrupando categorías poco frecuentes"):
        print(f"Procesando columna categórica: {col}")
        df = limpiar_y_agrupar(df, col, umbral=100)

    # Mostrar el estado del DataFrame después de agrupar categorías poco frecuentes
    print(f"Paso 2: Tamaño del DataFrame después de agrupar categorías: {df.shape}")

    # Convertir y expandir la columna 'auction_time'
    print(f"Paso 3: Expandiendo la columna 'auction_time'")
    categories_time, df = convertir_auction_time(df)
    all_categories.update(categories_time)
    print(f"Categorías extraídas de 'auction_time': {categories_time}")

    # Expandir la columna 'auction_list_0' usando expand_list_dummies
    print(f"Paso 4: Expandiendo la columna 'auction_list_0'")
    categories_auction, df = expand_list_dummies(
        df, 
        column='auction_list_0', 
        delimiter='|', 
        prefix=None, 
        suffix='auction_list_0'
    )
    all_categories.update(categories_auction)
    print(f"Categorías extraídas de 'auction_list_0': {categories_auction}")

    # Expandir la columna 'action_list_1' usando expand_list_dummies
    print(f"Paso 5: Expandiendo la columna 'action_list_1'")
    categories_action1, df = expand_list_dummies(
        df, 
        column='action_list_1', 
        delimiter='|', 
        prefix=None, 
        suffix='action_list_1'
    )
    all_categories.update(categories_action1)
    print(f"Categorías extraídas de 'action_list_1': {categories_action1}")

    # Expandir la columna 'action_list_2' usando expand_list_dummies
    print(f"Paso 6: Expandiendo la columna 'action_list_2'")
    categories_action2, df = expand_list_dummies(
        df, 
        column='action_list_2', 
        delimiter='|', 
        prefix=None, 
        suffix='action_list_2'
    )
    all_categories.update(categories_action2)
    print(f"Categorías extraídas de 'action_list_2': {categories_action2}")

    # Mostrar el tamaño del DataFrame antes de la limpieza adicional
    print(f"Paso 7: Tamaño del DataFrame antes de la limpieza adicional: {df.shape}")

    # Realizar limpieza adicional de los datos
    df = IntFloatToStr(df)

    # Tamaño final del DataFrame
    print(f"Paso 8: Tamaño final del DataFrame: {df.shape}")

    return all_categories, df

def ajustar_columnas_test(train_df, test_df):
    """
    Asegura que las columnas del conjunto de test coincidan con las del conjunto de entrenamiento,
    con la excepción de que 'Label' no se puede borrar de train y 'id' no se puede borrar de test.
    
    Si faltan columnas en el conjunto de test, se agregan con valor 0.
    Si sobran columnas en el conjunto de test que no están en el de entrenamiento, se eliminan.
    
    Parameters:
    - train_df: DataFrame del conjunto de entrenamiento (con las columnas transformadas)
    - test_df: DataFrame del conjunto de test (sin transformar)
    
    Returns:
    - test_df: DataFrame del conjunto de test con las columnas ajustadas.
    """
    # Encontrar las columnas que están en train pero no en test, excepto 'Label'
    missing_cols = set(train_df.columns) - set(test_df.columns) - {'Label'}

    # Encontrar las columnas que están en test pero no en train, excepto 'id'
    extra_cols = set(test_df.columns) - set(train_df.columns) - {'id'}

    # Agregar las columnas faltantes en el conjunto de test con valor 0
    for col in missing_cols:
        test_df[col] = 0

    # Eliminar las columnas extra del conjunto de test, excepto 'id'
    test_df = test_df.drop(columns=extra_cols)

    # Asegurarse de que las columnas estén en el mismo orden que en el conjunto de entrenamiento
    test_df = test_df[train_df.columns.difference(['Label']).tolist() + ['id']]

    return test_df

def ajustar_columnas_val(train_df, val_df):
    # Encontrar las columnas que están en train pero no en test, excepto 'Label'
    missing_cols = set(train_df.columns) - set(val_df.columns)

    # Encontrar las columnas que están en test pero no en train, excepto 'id'
    extra_cols = set(val_df.columns) - set(train_df.columns)

    # Agregar las columnas faltantes en el conjunto de test con valor 0
    for col in missing_cols:
        val_df[col] = 0

    # Eliminar las columnas extra del conjunto de test, excepto 'id'
    val_df = val_df.drop(columns=extra_cols)

    return val_df

# Función para manejar None en hiperparámetros en el nombre del archivo
def handle_none(value):
    return 'None' if value is None else str(value)


def concatenar(df1, df2):
    # Encontrar las columnas que están en df1 pero no en df2
    missing_cols = set(df1.columns) - set(df2.columns)

    # Encontrar las columnas que están en df2 pero no en df1
    extra_cols = set(df2.columns) - set(df1.columns)

    # Añadir las columnas faltantes en df2 con valores 0
    for col in missing_cols:
        df2[col] = 0

    # Añadir las columnas faltantes en df1 con valores 0
    for col in extra_cols:
        df1[col] = 0

    # Asegurarse de que ambas tengan las mismas columnas en el mismo orden
    df1 = df1[df2.columns]

    # Concatenar los DataFrames
    df_concat = pd.concat([df1, df2], ignore_index=True)

    return df_concat


## Data

In [3]:
val_data = pd.read_csv("train_data_cleaned_21.csv", dtype={'action_categorical_6': str})

train_data_cleaned_20 = pd.read_csv("train_data_cleaned_20.csv", dtype={'action_categorical_6': str})

In [4]:
# train_data_combined = concatenar(train_data_cleaned_21, train_data_cleaned_20)
train_data_combined = train_data_cleaned_20

In [ ]:
val_data = pd.read_csv("data/ctr_19.csv")

# Reducir el conjunto de datos mientras se mantiene la distribución de 'label'
val_data, _ = train_test_split(val_data, train_size=50000, stratify=val_data['Label'], random_state=random_state)

categories_val, val_data = data_cleaning(val_data)

val_data = ajustar_columnas_val(train_data_combined, val_data)

val_data = val_data[train_data_combined.columns]

## Caracteristicas principales

In [ ]:
# Imprimir la cantidad de filas del dataset combinado
print(f"Cantidad de filas en el dataset combinado: {train_data_combined.shape[0]}")
print(f"Cantidad de columnas en el dataset combinado: {train_data_combined.shape[1]}")

# Ver porcentaje de clics vs no clics en la columna Label
label_counts = train_data_combined['Label'].value_counts(normalize=True) * 100
print("\nPorcentaje de clics (1) y no clics (0):")
print(label_counts)

# Cantidad de clics (1) y no clics (0)
label_counts_abs = train_data_combined['Label'].value_counts()
print("\nCantidad de clics (1) y no clics (0):")
print(label_counts_abs)

In [ ]:
# Resumen estadístico de las características numéricas
print(train_data_combined.describe())

# Resumen de las características categóricas
print(train_data_combined.describe(include='object'))

In [ ]:
# Ver el porcentaje de valores faltantes por columna
missing_data = train_data_combined.isnull().mean() * 100
print(missing_data[missing_data > 0].sort_values(ascending=False))

In [ ]:
# Dividir los datos en numéricos y categóricos
numeric_features = train_data_combined.select_dtypes(include=['number']).columns.tolist()

categorical_features = train_data_combined.select_dtypes(include=['object']).columns.tolist()

## Columna por columna

In [ ]:
# Iterar sobre cada columna y mostrar los valores únicos
for column in train_data_combined.columns:
    unique_values = train_data_combined[column].unique()
    print(f"Columna: {column}")
    print(f"Valores únicos ({len(unique_values)}): {unique_values}")
    print("-" * 50)  # Separador entre columnas

In [ ]:
# Iterar sobre cada columna y mostrar los valores únicos y la cantidad de veces que aparecen
for column in train_data_combined.columns:
    print(f"Columna: {column}")
    print(f"Valores únicos y su frecuencia:")
    print(train_data_combined[column].value_counts())
    print("-" * 50)  # Separador entre columnas

In [ ]:
# Inicializar un conjunto vacío para almacenar los entity ids únicos
unique_entity_ids_2 = set()

# Recorrer cada fila de la columna 'action_list_2'
for lista in train_data_combined['action_list_2'].dropna():
    # Convertir el string de la lista a una lista real y agregar cada valor al conjunto
    unique_entity_ids_2.update(eval(lista))  # eval transforma la string en lista si es necesario

# Convertir el conjunto en una lista ordenada
unique_entity_ids_2 = sorted(unique_entity_ids_2)

# Mostrar los entity ids únicos
print(unique_entity_ids_2)
print(len(unique_entity_ids_2))

In [ ]:
# Inicializar un conjunto vacío para almacenar los entity ids únicos
unique_entity_ids = set()

# Recorrer cada fila de la columna 'action_list_1'
for lista in train_data_combined['action_list_1'].dropna():
    # Convertir el string de la lista a una lista real y agregar cada valor al conjunto
    unique_entity_ids.update(eval(lista))  # eval transforma la string en lista si es necesario

# Convertir el conjunto en una lista ordenada
unique_entity_ids = sorted(unique_entity_ids)

# Mostrar los entity ids únicos
print(unique_entity_ids)
print(len(unique_entity_ids))

In [ ]:
# Calcular la cantidad de valores NaN en cada columna
nan_counts = train_data_combined.isna().sum()

# Filtrar columnas que tienen al menos un NaN
nan_counts_filtered = nan_counts[nan_counts > 0]

# Mostrar las columnas con valores NaN de mayor a menor
print(nan_counts_filtered.sort_values(ascending=False))

## Data cleaning

In [ ]:
categories, train_data_combined = data_cleaning(train_data_combined)

# Encodings

In [ ]:
X_train = train_data_combined.drop(columns=['Label'])
y_train = train_data_combined['Label']

X_val = val_data.drop(columns=['Label'])
y_val = val_data['Label']

# Asegurarse de que X_train y X_val sean dataframes de pandas
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)

# Asegurarse de que y_train y y_val sean series de pandas
y_train = pd.Series(y_train)
y_val = pd.Series(y_val)

categories = [col for col in train_data_combined.columns if col.endswith(('_auction_list_0', '_action_list_1', '_action_list_2'))]

## Target Encoding

In [ ]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_combined.drop(columns='Label'),  # Características
    train_data_combined['Label'],               # Variable objetivo
    test_size=0.2,                              # 20% para validación
    stratify=train_data_combined['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Verificar la proporción de clases en el conjunto de entrenamiento y validación
print("Proporción en el conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True))

print("Proporción en el conjunto de validación:")
print(y_val.value_counts(normalize=True))


In [20]:
# Definir las columnas categóricas a codificar
categorical_features_to_encode = [col for col in X_train.select_dtypes(include=['object']).columns if col not in categories]

# Crear el Target Encoder para las columnas categóricas
target_encoder = ce.TargetEncoder(cols=categorical_features_to_encode)

# Crear el imputador
imputer = SimpleImputer(strategy='mean')

# Crear el clasificador de Random Forest
model_rf_te = RandomForestClassifier(random_state=random_state)

model_xgb_te = XGBClassifier(random_state=random_state)

# Crear el pipeline con TargetEncoder, imputación y Random Forest
pipeline_rf_te = Pipeline(steps=[
    ('target_encoder', target_encoder),  # Paso de Target Encoding
    ('imputer', imputer),                # Paso de imputación
    ('classifier', model_rf_te)          # Paso de Random Forest
])

pipeline_xgb_te = Pipeline(steps=[
    ('target_encoder', target_encoder),  # Paso de Target Encoding
    ('imputer', imputer),                # Paso de imputación
    ('classifier', model_xgb_te)         # Paso de Xgboost
])

### Busqueda de hiperparametros

#### Random Forest

##### Random search

In [ ]:
# Definir el espacio de búsqueda de hiperparámetros
param_dist = [
    {
        'classifier__bootstrap': [True],
        'classifier__max_samples': np.linspace(0.5, 1.0, num=10),
        'classifier__n_estimators': [100, 200, 300, 400, 500],
        'classifier__max_depth': [None, 10, 20, 30, 40, 50],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    },
    {
        'classifier__bootstrap': [False],
        'classifier__max_samples': [None],
        'classifier__n_estimators': [100, 200, 300, 400, 500],
        'classifier__max_depth': [None, 10, 20, 30, 40, 50],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
]

# Definir el RandomizedSearchCV para Random Forest
random_search_rf_te = RandomizedSearchCV(
    estimator=pipeline_rf_te,
    param_distributions=param_dist,
    n_iter=1,  # Aumenta el número de iteraciones para una búsqueda más exhaustiva
    scoring='roc_auc',
    cv=3,  # Número de folds en la validación cruzada
    verbose=2,
    random_state=random_state,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

In [ ]:
# Ajustar el RandomizedSearchCV al conjunto de entrenamiento de Random Forest
random_search_rf_te.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados para Random Forest:")
print(random_search_rf_te.best_params_)

# Predecir en el conjunto de validación con el mejor modelo encontrado
best_model_rf = random_search_rf_te.best_estimator_
y_pred_proba_rf = best_model_rf.predict_proba(X_val)[:, 1]

# Calcular el AUC en el conjunto de validación
auc_rf = roc_auc_score(y_val, y_pred_proba_rf)
print(f"AUC en el conjunto de validación para Random Forest: {auc_rf:.4f}")

#### XGBoost

##### Hyperopt

In [21]:
# Espacio de búsqueda para los hiperparámetros de XGBoost
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]), 
    'gamma': hp.uniform('gamma', 0, 0.5),
    'min_child_weight': hp.uniform('min_child_weight', 1, 10),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0.5, 2),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),  # L1 regularización
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),  # L2 regularización
    'max_delta_step': hp.uniform('max_delta_step', 0, 10),  # Para datos desbalanceados
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),  # Muestreo a nivel de split
    'colsample_bynode': hp.uniform('colsample_bynode', 0.5, 1),  # Muestreo por nodo
    'grow_policy': hp.choice('grow_policy', ['depthwise', 'lossguide']),  # Estrategia de crecimiento
    'tree_method': hp.choice('tree_method', ['auto', 'approx', 'hist'])  # Métodos de construcción del árbol
}

# Función objetivo para Hyperopt
def objective_xgb_te(params):

    params_mapped = {
        'max_depth': params['max_depth'],  # 'max_depth' ya está bien indexado
        'learning_rate': params['learning_rate'],
        'n_estimators': params['n_estimators'],  # Mapear n_estimators
        'gamma': params['gamma'],
        'min_child_weight': int(params['min_child_weight']),  # Asegurar que sea entero
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'scale_pos_weight': params['scale_pos_weight'],
        'reg_alpha': params['reg_alpha'],  # L1 regularización
        'reg_lambda': params['reg_lambda'],  # L2 regularización
        'max_delta_step': params['max_delta_step'],  # Para datos desbalanceados
        'colsample_bylevel': params['colsample_bylevel'],  # Muestreo a nivel de split
        'colsample_bynode': params['colsample_bynode'],  # Muestreo por nodo
        'grow_policy': params['grow_policy'],  # Estrategia de crecimiento
        'tree_method': params['tree_method']  # Método de construcción del árbol
    }

    # Definir el modelo con los hiperparámetros actuales
    model_xgb_te = XGBClassifier(
        max_depth=int(params_mapped['max_depth']),
        learning_rate=params_mapped['learning_rate'],
        n_estimators=int(params_mapped['n_estimators']),
        gamma=params_mapped['gamma'],
        min_child_weight=params_mapped['min_child_weight'],
        subsample=params_mapped['subsample'],
        colsample_bytree=params_mapped['colsample_bytree'],
        scale_pos_weight=params_mapped['scale_pos_weight'],
        reg_alpha=params_mapped['reg_alpha'],  # L1 regularización
        reg_lambda=params_mapped['reg_lambda'],  # L2 regularización
        max_delta_step=params_mapped['max_delta_step'],  # Para datos desbalanceados
        colsample_bylevel=params_mapped['colsample_bylevel'],  # Muestreo a nivel de split
        colsample_bynode=params_mapped['colsample_bynode'],  # Muestreo por nodo
        grow_policy=params_mapped['grow_policy'],  # Estrategia de crecimiento
        tree_method=params_mapped['tree_method'],  # Método de construcción del árbol
        use_label_encoder=False,  # Evitar advertencias en versiones más recientes de XGBoost
        eval_metric='auc',  # Métrica de evaluación
        random_state=random_state  # Para reproducibilidad
    )

    pipeline_xgb_te = Pipeline(steps=[
        ('target_encoder', target_encoder),  # Paso de Target Encoding
        ('imputer', imputer),                # Paso de imputación
        ('classifier', model_xgb_te)         # Paso de Xgboost
    ])
    
    # Entrenar el modelo con el conjunto de entrenamiento preprocesado para XGBoost
    pipeline_xgb_te.fit(X_train, y_train)
    
    # Hacer predicciones en el conjunto de validación preprocesado para XGBoost
    y_pred_proba = pipeline_xgb_te.predict_proba(X_val)[:, 1]
    
    # Calcular el AUC en el conjunto de validación
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params_mapped}, AUC: {auc:.4f}")
    
    # Retornar el valor negativo del AUC ya que Hyperopt minimiza por defecto
    return {'loss': 1 - auc, 'status': STATUS_OK}


In [ ]:
# Ejecutar la optimización
trials = Trials()

best_xgb = fmin(
    fn=objective_xgb_te,
    space=space,
    algo=tpe.suggest,
    max_evals=5,
    trials=trials,
    rstate=np.random.default_rng(random_state)  # Asegurar reproducibilidad
)

# No es necesario volver a mapear los hiperparámetros aquí, ya se hizo dentro de la función objetivo
print("Mejores hiperparámetros para XGBoost:")
print(best_xgb)

### Prueba de hiperparametros

In [ ]:
# Cargar los datos de testeo
test_data = pd.read_csv("data/ctr_test.csv")

test_data_cleaned = pd.read_csv('test_data_cleaned.csv')

# Ajustar las columnas de testeo para que coincidan con las de entrenamiento
test_data_cleaned = ajustar_columnas_test(train_data_combined, test_data_cleaned)

test_data_cleaned = test_data_cleaned[X_train.columns]

#### Random Forest

##### Random search

In [ ]:
# Convertir los hiperparámetros a los tipos correctos
random_search_rf_te.best_params_['classifier__n_estimators'] = int(random_search_rf_te.best_params_['classifier__n_estimators'])
random_search_rf_te.best_params_['classifier__min_samples_split'] = int(random_search_rf_te.best_params_['classifier__min_samples_split'])
random_search_rf_te.best_params_['classifier__min_samples_leaf'] = int(random_search_rf_te.best_params_['classifier__min_samples_leaf'])
random_search_rf_te.best_params_['classifier__max_depth'] = int(random_search_rf_te.best_params_['classifier__max_depth']) if random_search_rf_te.best_params_['classifier__max_depth'] is not None else None

# Reconstruir el modelo con los mejores hiperparámetros
best_model_rf = RandomForestClassifier(
    n_estimators=random_search_rf_te.best_params_['classifier__n_estimators'],
    max_depth=random_search_rf_te.best_params_['classifier__max_depth'],  # Asegurarse de que max_depth sea un entero o None
    min_samples_split=random_search_rf_te.best_params_['classifier__min_samples_split'],
    min_samples_leaf=random_search_rf_te.best_params_['classifier__min_samples_leaf'],
    bootstrap=random_search_rf_te.best_params_['classifier__bootstrap'],
    random_state=random_state,
    max_samples=random_search_rf_te.best_params_['classifier__max_samples']
)

# Crear un nuevo pipeline reutilizando el preprocesador original y el mejor modelo
pipeline_rf_te_best = Pipeline(steps=[
    ('target_encoder', target_encoder),
    ('imputer', imputer),
    ('classifier', best_model_rf)
])

# Entrenar el pipeline con los datos de entrenamiento
pipeline_rf_te_best.fit(X_train, y_train)

In [ ]:
# Predecir en el conjunto de testeo
y_preds_rf_test = pipeline_rf_te_best.predict_proba(test_data_cleaned)[:, 1]

# Crear el archivo de envío
submission_df_rf = pd.DataFrame({"id": test_data["id"], "Label": y_preds_rf_test})
submission_df_rf["id"] = submission_df_rf["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "random_forest"
file_name_rf = (
    f"random_forest_predictions_n_estimators_{random_search_rf_te.best_params_['classifier__n_estimators']}_"
    f"max_depth_{random_search_rf_te.best_params_['classifier__max_depth']}_"
    f"min_samples_split_{random_search_rf_te.best_params_['classifier__min_samples_split']}_"
    f"max_samples_{random_search_rf_te.best_params_['classifier__max_samples']}_"
    f"bootstrap_{random_search_rf_te.best_params_['classifier__bootstrap']}_"
    f"min_samples_leaf_{random_search_rf_te.best_params_['classifier__min_samples_leaf']}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")

## One Hot Encoding

In [11]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_combined.drop(columns='Label'),  # Características
    train_data_combined['Label'],               # Variable objetivo
    test_size=0.2,                              # 20% para validación
    stratify=train_data_combined['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Asegurarse de que X_train y X_val sean dataframes de pandas
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)

# Asegurarse de que y_train y y_val sean series de pandas
y_train = pd.Series(y_train)
y_val = pd.Series(y_val)

In [ ]:
categories = [col for col in train_data_combined.columns if col.endswith(('_auction_list_0', '_action_list_1', '_action_list_2'))]

# Definir las columnas categóricas y numéricas
categorical_features_to_encode = [col for col in X_train.select_dtypes(include=['object']).columns if col not in categories]
numeric_features = X_train.select_dtypes(include=['number']).columns.tolist()

# Preprocesador común para imputación y codificación
common_preprocessor = ColumnTransformer(
    transformers=[
        # Imputar valores numéricos con la media
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        
        # Imputar valores categóricos con 'Desconocido' y aplicar One-Hot Encoding
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Desconocido')),
            ('onehot', OneHotEncoder(drop='first', sparse_output=True, handle_unknown='ignore'))
        ]), categorical_features_to_encode)
    ],
    remainder='drop'  # Excluir columnas no especificadas
)

model_rf = RandomForestClassifier(random_state=random_state)

# Definir el modelo XGBoost
model_xgb = XGBClassifier(
    random_state=random_state, 
    use_label_encoder=False, 
    eval_metric='auc'
)

# Pipeline para XGBoost (mantiene matrices dispersas)
pipeline_xgb = Pipeline(steps=[
    ('preprocessor', common_preprocessor),
    ('classifier', model_xgb),
])

# Pipeline para Random Forest (convierte a denso y optimiza tipos de datos)
pipeline_rf = Pipeline(steps=[
    ('preprocessor', common_preprocessor),
    ('to_dense', FunctionTransformer(lambda x: x.tocsr().astype('float32').toarray(), accept_sparse=True)),
    ('classifier', model_rf),
])

### Busqueda de hiperparametros

#### Random Forest

##### Random search

In [9]:
# Definir el espacio de búsqueda de hiperparámetros
param_dist = [
    {
        'classifier__bootstrap': [True],
        'classifier__max_samples': np.linspace(0.5, 1.0, num=10),
        'classifier__n_estimators': [100, 200, 300, 400, 500],
        'classifier__max_depth': [None, 10, 20, 30, 40, 50],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    },
    {
        'classifier__bootstrap': [False],
        'classifier__max_samples': [None],
        'classifier__n_estimators': [100, 200, 300, 400, 500],
        'classifier__max_depth': [None, 10, 20, 30, 40, 50],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
]

# Definir el RandomizedSearchCV para Random Forest
random_search_rf = RandomizedSearchCV(
    estimator=pipeline_rf,
    param_distributions=param_dist,
    n_iter=1,  # Aumenta el número de iteraciones para una búsqueda más exhaustiva
    scoring='roc_auc',
    cv=3,  # Número de folds en la validación cruzada
    verbose=2,
    random_state=random_state,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

In [ ]:
# Ajustar el RandomizedSearchCV al conjunto de entrenamiento de Random Forest
random_search_rf.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados para Random Forest:")
print(random_search_rf.best_params_)

# Predecir en el conjunto de validación con el mejor modelo encontrado
best_model_rf = random_search_rf.best_estimator_
y_pred_proba_rf = best_model_rf.predict_proba(X_val)[:, 1]

# Calcular el AUC en el conjunto de validación
auc_rf = roc_auc_score(y_val, y_pred_proba_rf)
print(f"AUC en el conjunto de validación para Random Forest: {auc_rf:.4f}")

##### Hyperopt

In [ ]:
# Definir el espacio de búsqueda de hiperparámetros
space_rf = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [None, 10, 20, 30, 40, 50]),
    'max_samples': hp.uniform('max_samples', 0.5, 1.0),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),
    'bootstrap': hp.choice('bootstrap', [True, False])
}

# Función objetivo para Hyperopt
def objective_rf(params):
    # Asegurarse de que 'max_samples' solo se usa si 'bootstrap' es True
    if not params['bootstrap']:
        params.pop('max_samples', None)  # No se puede utilizar max_samples si bootstrap es False
    
    # Crear el clasificador con los parámetros actuales
    model_rf = RandomForestClassifier(**params, random_state=random_state)
    
    # Crear el pipeline con el preprocesador común y el clasificador
    pipeline_rf = Pipeline(steps=[
        ('preprocessor', common_preprocessor),
        ('to_dense', FunctionTransformer(lambda x: x.tocsr().astype('float32').toarray(), accept_sparse=True)),
        ('classifier', model_rf),
    ])
    
    # Entrenar el pipeline en el conjunto de entrenamiento
    pipeline_rf.fit(X_train, y_train)
    
    # Predecir probabilidades en el conjunto de validación
    y_pred_proba = pipeline_rf.predict_proba(X_val)[:, 1]
    
    # Calcular el AUC en el conjunto de validación
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")
    
    # Retornar el valor negativo del AUC ya que Hyperopt minimiza por defecto
    return {'loss': 1 - auc, 'status': STATUS_OK}

In [ ]:
# Ejecutar la optimización
trials = Trials()  # Para guardar información sobre cada iteración
best_rf = fmin(
    fn=objective_rf, 
    space=space_rf, 
    algo=tpe.suggest, 
    max_evals=10,  # Número de evaluaciones aumentado para una búsqueda más exhaustiva
    trials=trials,
    rstate=np.random.default_rng(random_state)  # Asegurar reproducibilidad
)

# Definir las opciones de los parámetros que usan hp.choice
max_depth_options = [None, 10, 20, 30, 40, 50]
bootstrap_options = [True, False]
max_samples_options = [None] + list(np.linspace(0.5, 1.0, num=6))  # [None, 0.5, 0.6, ..., 1.0]
min_samples_split_options = [2, 5, 10]

if not bootstrap_options[best_rf['bootstrap']]:
    best_rf['max_samples'] = None  # No se puede utilizar max_samples si bootstrap es False

# Mapear los índices de hp.choice a los valores reales
best_params_rf_mapped = {
    'classifier__n_estimators': int(best_rf['n_estimators']),
    'classifier__max_depth': max_depth_options[best_rf['max_depth']],
    'classifier__max_samples': best_rf['max_samples'],
    'classifier__min_samples_split': min_samples_split_options[best_rf['min_samples_split']],
    'classifier__min_samples_leaf': int(best_rf['min_samples_leaf']),
    'classifier__bootstrap': bootstrap_options[best_rf['bootstrap']]
}

print("Mejores hiperparámetros mapeados para Random Forest:")
print(best_params_rf_mapped)

#### XGBoost

##### Random search

In [ ]:
# Definir el espacio de búsqueda de hiperparámetros para XGBoost
param_dist_xgb = {
    'classifier__n_estimators': [100, 200, 300, 400, 500],
    'classifier__max_depth': [3, 5, 10, 15, 20],  # Profundidad máxima del árbol
    'classifier__learning_rate': np.linspace(0.01, 0.3, 10),  # Tasa de aprendizaje
    'classifier__subsample': np.linspace(0.5, 1.0, num=5),  # Fracción de muestras a usar para cada árbol
    'classifier__colsample_bytree': np.linspace(0.5, 1.0, num=5),  # Fracción de características a usar para cada árbol
    'classifier__gamma': [0, 0.1, 0.2, 0.3],  # Regularización
    'classifier__reg_alpha': [0, 0.01, 0.1, 1],  # Regularización L1
    'classifier__reg_lambda': [1, 1.5, 2, 3]  # Regularización L2
}

# Definir el RandomizedSearchCV para XGBoost
random_search_xgb = RandomizedSearchCV(
    estimator=pipeline_xgb,
    param_distributions=param_dist_xgb,
    n_iter=3,  # Aumenta el número de combinaciones de hiperparámetros a probar
    scoring='roc_auc',  # Métrica para evaluar
    cv=3,  # Número de folds en la validación cruzada
    verbose=2,
    random_state=random_state,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

In [ ]:
# Ajustar el modelo a los datos de entrenamiento preprocesados para XGBoost
random_search_xgb.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados para XGBoost:")
print(random_search_xgb.best_params_)

# Predecir en el conjunto de validación con el mejor modelo encontrado
best_model_xgb = random_search_xgb.best_estimator_
y_pred_proba_xgb = best_model_xgb.predict_proba(X_val)[:, 1]

# Calcular el AUC en el conjunto de validación
auc_xgb = roc_auc_score(y_val, y_pred_proba_xgb)
print(f"AUC en el conjunto de validación para XGBoost: {auc_xgb:.4f}")

##### Hyperopt

In [13]:
# Espacio de búsqueda para los hiperparámetros de XGBoost
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]), 
    'gamma': hp.uniform('gamma', 0, 0.5),
    'min_child_weight': hp.uniform('min_child_weight', 1, 10),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0.5, 2),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),  # L1 regularización
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),  # L2 regularización
    'max_delta_step': hp.uniform('max_delta_step', 0, 10),  # Para datos desbalanceados
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),  # Muestreo a nivel de split
    'colsample_bynode': hp.uniform('colsample_bynode', 0.5, 1),  # Muestreo por nodo
    'grow_policy': hp.choice('grow_policy', ['depthwise', 'lossguide']),  # Estrategia de crecimiento
    'tree_method': hp.choice('tree_method', ['auto', 'approx', 'hist'])  # Métodos de construcción del árbol
}

# Función objetivo para Hyperopt
def objective_xgb(params):

    params_mapped = {
        'max_depth': params['max_depth'],  # 'max_depth' ya está bien indexado
        'learning_rate': params['learning_rate'],
        'n_estimators': params['n_estimators'],  # Mapear n_estimators
        'gamma': params['gamma'],
        'min_child_weight': int(params['min_child_weight']),  # Asegurar que sea entero
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'scale_pos_weight': params['scale_pos_weight'],
        'reg_alpha': params['reg_alpha'],  # L1 regularización
        'reg_lambda': params['reg_lambda'],  # L2 regularización
        'max_delta_step': params['max_delta_step'],  # Para datos desbalanceados
        'colsample_bylevel': params['colsample_bylevel'],  # Muestreo a nivel de split
        'colsample_bynode': params['colsample_bynode'],  # Muestreo por nodo
        'grow_policy': params['grow_policy'],  # Estrategia de crecimiento
        'tree_method': params['tree_method']  # Método de construcción del árbol
    }

    # Definir el modelo con los hiperparámetros actuales
    model_xgb = XGBClassifier(
        max_depth=int(params_mapped['max_depth']),
        learning_rate=params_mapped['learning_rate'],
        n_estimators=int(params_mapped['n_estimators']),
        gamma=params_mapped['gamma'],
        min_child_weight=params_mapped['min_child_weight'],
        subsample=params_mapped['subsample'],
        colsample_bytree=params_mapped['colsample_bytree'],
        scale_pos_weight=params_mapped['scale_pos_weight'],
        reg_alpha=params_mapped['reg_alpha'],  # L1 regularización
        reg_lambda=params_mapped['reg_lambda'],  # L2 regularización
        max_delta_step=params_mapped['max_delta_step'],  # Para datos desbalanceados
        colsample_bylevel=params_mapped['colsample_bylevel'],  # Muestreo a nivel de split
        colsample_bynode=params_mapped['colsample_bynode'],  # Muestreo por nodo
        grow_policy=params_mapped['grow_policy'],  # Estrategia de crecimiento
        tree_method=params_mapped['tree_method'],  # Método de construcción del árbol
        use_label_encoder=False,  # Evitar advertencias en versiones más recientes de XGBoost
        eval_metric='auc',  # Métrica de evaluación
        random_state=random_state  # Para reproducibilidad
    )

    pipeline_xgb = Pipeline(steps=[
        ('preprocessor', common_preprocessor),
        ('classifier', model_xgb),
    ])
    
    # Entrenar el modelo con el conjunto de entrenamiento preprocesado para XGBoost
    pipeline_xgb.fit(X_train, y_train)
    
    # Hacer predicciones en el conjunto de validación preprocesado para XGBoost
    y_pred_proba = pipeline_xgb.predict_proba(X_val)[:, 1]
    
    # Calcular el AUC en el conjunto de validación
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params_mapped}, AUC: {auc:.4f}")
    
    # Retornar el valor negativo del AUC ya que Hyperopt minimiza por defecto
    return {'loss': 1 - auc, 'status': STATUS_OK}


In [ ]:
# Ejecutar la optimización
trials = Trials()

best_xgb = fmin(
    fn=objective_xgb,
    space=space,
    algo=tpe.suggest,
    max_evals=5,
    trials=trials,
    rstate=np.random.default_rng(random_state)  # Asegurar reproducibilidad
)

# No es necesario volver a mapear los hiperparámetros aquí, ya se hizo dentro de la función objetivo
print("Mejores hiperparámetros para XGBoost:")
print(best_xgb)

### Pueba de Hiperparametros

In [15]:
# Cargar los datos de testeo
test_data = pd.read_csv("data/ctr_test.csv")

X_test = pd.read_csv('test_data_cleaned.csv', dtype={'action_categorical_6': str})

# # Ajustar las columnas de testeo para que coincidan con las de entrenamiento
# X_test = ajustar_columnas_test(X_train, X_test)

# X_test = X_test[X_train.columns]


In [ ]:
X_test = X_test[['auction_bidfloor', 'creative_height', 'creative_width', 'tarde', 'noche']]

X_test = X_test[X_train.columns]

print(X_test.shape[1])
print(X_train.shape[1])

#### Random Forest

##### Random search

In [ ]:
# Convertir los hiperparámetros a los tipos correctos
random_search_rf.best_params_['classifier__n_estimators'] = int(random_search_rf.best_params_['classifier__n_estimators'])
random_search_rf.best_params_['classifier__min_samples_split'] = int(random_search_rf.best_params_['classifier__min_samples_split'])
random_search_rf.best_params_['classifier__min_samples_leaf'] = int(random_search_rf.best_params_['classifier__min_samples_leaf'])
random_search_rf.best_params_['classifier__max_depth'] = int(random_search_rf.best_params_['classifier__max_depth']) if random_search_rf.best_params_['classifier__max_depth'] is not None else None

# Reconstruir el modelo con los mejores hiperparámetros
best_model_rf = RandomForestClassifier(
    n_estimators=random_search_rf.best_params_['classifier__n_estimators'],
    max_depth=random_search_rf.best_params_['classifier__max_depth'],  # Asegurarse de que max_depth sea un entero o None
    min_samples_split=random_search_rf.best_params_['classifier__min_samples_split'],
    min_samples_leaf=random_search_rf.best_params_['classifier__min_samples_leaf'],
    bootstrap=random_search_rf.best_params_['classifier__bootstrap'],
    random_state=random_state,
    max_samples=random_search_rf.best_params_['classifier__max_samples']
)

# Crear un nuevo pipeline reutilizando el preprocesador original y el mejor modelo
rf_pipeline = Pipeline(steps=[
    ('preprocessor', common_preprocessor),  # Reutilizamos el preprocesador definido previamente
    ('to_dense', FunctionTransformer(lambda x: x.tocsr().astype('float32').toarray(), accept_sparse=True)),
    ('classifier', best_model_rf),
])

# Entrenar el pipeline con los datos de entrenamiento
rf_pipeline.fit(X_train, y_train)

In [ ]:
# Predecir en el conjunto de testeo
y_preds_rf_test = rf_pipeline.predict_proba(X_test)[:, 1]

# Crear el archivo de envío
submission_df_rf = pd.DataFrame({"id": test_data["id"], "Label": y_preds_rf_test})
submission_df_rf["id"] = submission_df_rf["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "random_forest"
file_name_rf = (
    f"random_forest_predictions_n_estimators_{random_search_rf.best_params_['classifier_n_estimators']}_"
    f"max_depth_{random_search_rf.best_params_['classifier_max_depth']}_"
    f"min_samples_split_{random_search_rf.best_params_['classifier_min_samples_split']}_"
    f"max_samples_{random_search_rf.best_params_['classifier_max_samples']}_"
    f"bootstrap_{random_search_rf.best_params_['classifier_bootstrap']}_"
    f"min_samples_leaf_{random_search_rf.best_params_['classifier_min_samples_leaf']}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")

##### Hyperopt

In [ ]:
best_model_rf = RandomForestClassifier(
    n_estimators=best_params_rf_mapped['classifier__n_estimators'],
    max_depth=best_params_rf_mapped['classifier__max_depth'],
    max_samples=best_params_rf_mapped['classifier__max_samples'],
    min_samples_split=best_params_rf_mapped['classifier__min_samples_split'],
    min_samples_leaf=best_params_rf_mapped['classifier__min_samples_leaf'],
    bootstrap=best_params_rf_mapped['classifier__bootstrap'],
    random_state=random_state
)

# Crear un nuevo pipeline reutilizando el preprocesador original y el mejor modelo
rf_pipeline = Pipeline(steps=[
    ('preprocessor', common_preprocessor),  # Reutilizamos el preprocesador definido previamente
    ('to_dense', FunctionTransformer(lambda x: x.tocsr().astype('float32').toarray(), accept_sparse=True)),
    ('classifier', best_model_rf),
])

rf_pipeline.fit(X_train, y_train)

In [ ]:
# Generar predicciones de probabilidad en el conjunto de testeo preprocesado
y_preds_rf_test = rf_pipeline.predict_proba(X_test)[:, 1]

# Crear el archivo de envío
submission_df_xgb = pd.DataFrame({"id": test_data["id"], "Label": y_preds_rf_test})
submission_df_xgb["id"] = submission_df_xgb["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "random_forest"
file_name_rf = (
    f"random_forest_predictions_n_estimators_{best_params_rf_mapped['classifier__n_estimators']}_"
    f"max_depth_{handle_none(best_params_rf_mapped['classifier__max_depth'])}_"
    f"min_samples_split_{best_params_rf_mapped['classifier__min_samples_split']}_"
    f"max_samples_{handle_none(best_params_rf_mapped['classifier__max_samples'])}_"
    f"bootstrap_{best_params_rf_mapped['classifier__bootstrap']}_"
    f"min_samples_leaf_{best_params_rf_mapped['classifier__min_samples_leaf']}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")

#### XGboost

##### Random search

In [ ]:
# Convertir los hiperparámetros a los tipos correctos
random_search_xgb.best_params_['classifier__n_estimators'] = int(random_search_xgb.best_params_['classifier__n_estimators'])
random_search_xgb.best_params_['classifier__min_samples_split'] = int(random_search_xgb.best_params_['classifier__min_samples_split'])
random_search_xgb.best_params_['classifier__min_samples_leaf'] = int(random_search_xgb.best_params_['classifier__min_samples_leaf'])
random_search_xgb.best_params_['classifier__max_depth'] = int(random_search_xgb.best_params_['classifier__max_depth']) if random_search_xgb.best_params_['classifier__max_depth'] is not None else None

# Reconstruir el modelo con los mejores hiperparámetros
best_model_xgb = RandomForestClassifier(
    n_estimators=random_search_xgb['classifier__n_estimators'],
    max_depth=random_search_xgb['classifier__max_depth'],
    learning_rate=random_search_xgb['classifier__learning_rate'],
    subsample=random_search_xgb['classifier__subsample'],
    colsample_bytree=random_search_xgb['classifier__colsample_bytree'],
    min_child_weight=random_search_xgb['classifier__min_child_weight'],
    gamma=random_search_xgb['classifier__gamma'],
    reg_alpha=random_search_xgb['classifier__reg_alpha'],
    reg_lambda=random_search_xgb['classifier__reg_lambda'],
    random_state=random_state,
    use_label_encoder=False,
    eval_metric="auc"  # Establecer AUC como métrica de evaluación
)

# Crear un nuevo pipeline reutilizando el preprocesador original y el mejor modelo
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', common_preprocessor),  # Reutilizamos el preprocesador definido previamente
    ('to_dense', FunctionTransformer(lambda x: x.tocsr().astype('float32').toarray(), accept_sparse=True)),
    ('classifier', best_model_xgb),
])

# Entrenar el pipeline con los datos de entrenamiento
xgb_pipeline.fit(X_train, y_train)

In [ ]:
# Predecir en el conjunto de testeo
y_preds_rf_test = xgb_pipeline.predict_proba(X_test)[:, 1]

# Crear el archivo de envío
submission_df_rf = pd.DataFrame({"id": test_data["id"], "Label": y_preds_rf_test})
submission_df_rf["id"] = submission_df_rf["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "random_forest"
file_name_rf = (
    f"xgboost_predictions_n_estimators_{random_search_xgb['classifier__n_estimators']}_"
    f"max_depth_{random_search_xgb['classifier__max_depth']}_"
    f"learning_rate_{random_search_xgb['lclassifier__earning_rate']}_"
    f"subsample_{random_search_xgb['classifier__subsample']}_"
    f"colsample_bytree_{random_search_xgb['classifier__colsample_bytree']}_"
    f"min_child_weight_{random_search_xgb['classifier__min_child_weight']}_"
    f"gamma_{random_search_xgb['classifier__gamma']}_"
    f"reg_alpha_{random_search_xgb['classifier__reg_alpha']}_"
    f"reg_lambda_{random_search_xgb['classifier__reg_lambda']}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")

##### Hyperopt

In [ ]:
# Indices
tree_method_options = ['auto', 'exact', 'approx', 'hist']
grow_policy_options = ['depthwise', 'lossguide']


# Reconstruir el modelo de XGBoost con los mejores hiperparámetros
best_model_xgb = xgb.XGBClassifier(
    n_estimators=best_xgb['n_estimators'],
    max_depth=best_xgb['max_depth'],
    learning_rate=best_xgb['learning_rate'],
    subsample=best_xgb['subsample'],
    colsample_bytree=best_xgb['colsample_bytree'],
    min_child_weight=best_xgb['min_child_weight'],
    gamma=best_xgb['gamma'],
    reg_alpha=best_xgb['reg_alpha'],  # L1 regularización
    reg_lambda=best_xgb['reg_lambda'],  # L2 regularización
    max_delta_step=best_xgb['max_delta_step'],  # Para datos desbalanceados
    colsample_bylevel=best_xgb['colsample_bylevel'],  # Muestreo a nivel de split
    colsample_bynode=best_xgb['colsample_bynode'],  # Muestreo por nodo
    grow_policy=grow_policy_options[best_xgb['grow_policy']],  # Estrategia de crecimiento
    tree_method=tree_method_options[best_xgb['tree_method']],  # Método de construcción del árbol
    random_state=random_state,
    use_label_encoder=False,  # Evitar advertencias en versiones más recientes de XGBoost
    eval_metric="auc"  # Establecer AUC como métrica de evaluación
)

# Crear un nuevo pipeline reutilizando el preprocesador original y el mejor modelo
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', common_preprocessor),  # Reutilizamos el preprocesador definido previamente
    ('classifier', best_model_xgb),
])

# Entrenar el modelo con los datos de entrenamiento
xgb_pipeline.fit(X_train, y_train)

In [ ]:
# Predecir en el conjunto de testeo
y_preds_xgb_test = xgb_pipeline.predict_proba(X_test)[:, 1]

# Crear el archivo de envío
submission_df_xgb = pd.DataFrame({"id": test_data["id"], "Label": y_preds_xgb_test})
submission_df_xgb["id"] = submission_df_xgb["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros
file_name_xgb = (
    f"xgboost_preds_ne_{best_xgb['n_estimators']}_"
    f"md_{handle_none(best_xgb['max_depth'])}_"
    f"lr_{round(best_xgb['learning_rate'], 2)}_"
    f"ss_{round(best_xgb['subsample'], 2)}_"
    f"csb_{round(best_xgb['colsample_bytree'], 2)}_"
    f"mcw_{round(best_xgb['min_child_weight'], 2)}_"
    f"gamma_{round(best_xgb['gamma'], 2)}_"
    f"ra_{round(best_xgb['reg_alpha'], 2)}_"
    f"rl_{round(best_xgb['reg_lambda'], 2)}_"
    f"mds_{round(best_xgb['max_delta_step'], 2)}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_xgb.to_csv(os.path.join("submits", file_name_xgb), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_xgb}")